In [ ]:
import cv2
import numpy as np

image = cv2.imread(r'C:\Users\Jai Dev Dwawdi\Desktop\12.jpg')
cv2.imshow('input image',image)

gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

edged = cv2.Canny(gray,30,200)
cv2.imshow('canny edges',edged)

contours, hierarchy = cv2.findContours(edged.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
cv2.imshow('canny edges after contouring',edged)

print(" numbers of contours" + str(len(contours)))

cv2.drawContours(image, contours, -1, (0,255,0),3)

cv2.imshow('contours',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
print(contours)

Contour Sorting

In [ ]:
import cv2
import numpy as np


def get_contour_areas(contours):
    all_areas = []
    for item in contours:
        area = cv2.contourArea(item)
        all_areas.append(area)
    return all_areas

image = cv2.imread(r'C:\Users\Jai Dev Dwawdi\Desktop\12.jpg')
original_image = image
x = get_contour_areas(contours)
print (x)

sorted_contours = sorted(contours, key = cv2.contourArea, reverse = True)
y = get_contour_areas(sorted_contours)
print (y)

for item in sorted_contours:
    cv2.drawContours(original_image,[item],-1,(255,0,0),3)
    cv2.waitKey(0)
    cv2.imshow('contours by area', original_image)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

centroid method

In [ ]:
import cv2
import numpy as np

def x_cord_contour(contours):
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))
    
def label_contour_center(image,c):
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    cv2.circle(image,(cx,cy),10,(0,0,255),-1)
    return image

image = cv2.imread(r'C:\Users\Jai Dev Dwawdi\Desktop\12.jpg')
original_image = image.copy()

for (i ,c) in enumerate(contours):
    orig = label_contour_center(image, c)

cv2.imshow('contour centers', image)
cv2.waitKey(0)

contours_left_to_right = sorted(contours, key = x_cord_contour, reverse= False)

for (i,c) in enumerate(contours_left_to_right):
    cv2.drawContours(original_image,[c],-1,(0,0,255),3)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(original_image, str(i+1), (cx,cy), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('left to right contour', original_image)
    cv2.waitKey(0)
    (x,y,w,h) = cv2.boundingRect(c)
    
cv2.destroyAllWindows()
    

Approximating contours

In [2]:
import cv2
import numpy as np

image = cv2.imread(r'C:\Users\Jai Dev Dwawdi\Desktop\12.jpg')
orig_image = image.copy()
cv2.imshow('original image', orig_image)
cv2.waitKey(0)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

contours, hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)

for item in contours:
    x,y,w,h = cv2.boundingRect(item)
    cv2.rectangle(orig_image,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.imshow('bounding rectangles',orig_image)

cv2.waitKey(0)

for item in contours:
    accuracy = 0.03* cv2.arcLength(item,True)
    approx = cv2.approxPolyDP(item,accuracy, True)
    cv2.drawContours(image,[approx],0,(0,255,0),2)
    cv2.imshow('approx poly dp', image)
    
cv2.waitKey(0)
cv2.destroyAllWindows()




convex hull

In [3]:
import cv2
import numpy as np

image = cv2.imread(r'C:\Users\Jai Dev Dwawdi\Desktop\12.jpg')
orig_image = image.copy()
cv2.imshow('original image', orig_image)
cv2.waitKey(0)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

contours, hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)

n = len(contours) - 1
contours = sorted(contours, key = cv2.contourArea, reverse=False)[:n]

for item in contours:
    hull = cv2.convexHull(item)
    cv2.drawContours(image,[hull],0,(0,255,0),2)
    cv2.imshow('convex hull', image)
    
cv2.waitKey(0)
cv2.destroyAllWindows()


Match


In [19]:
import cv2
import numpy as np

temp = cv2.imread(r'C:\Users\Jai Dev Dwawdi\Desktop\12.jpg')
template = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
cv2.imshow('template',template)
cv2.waitKey(0)

target = cv2.imread(r'C:\Users\Jai Dev Dwawdi\Desktop\bs.jpg')
target_gray = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)

ret, thres1 = cv2.threshold(template,127,255,0)
ret, thres2 = cv2.threshold(target_gray,127,255,0)

contours, hierarchy = cv2.findContours(thres1,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
sorted_contours = sorted(contours, key = cv2.contourArea, reverse = True)

template_contour = contours[1]

contours, hierarchy = cv2.findContours(thres2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

for item in contours:
    match = cv2.matchShapes(template_contour, item, 1, 0.0)
    print (match)
    
    if match < 0.15:
        closest_contour = item
        break

if closest_contour is not None:
    cv2.drawContours(target,[closest_contour], -1, (0,255,0), 3)
else:
    print("No closest contour found")

cv2.imshow('Output',target)
cv2.waitKey()
cv2.destroyAllWindows()
    
                    

0.0
